Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

root = "/content/gdrive/My Drive/Escola/Master/Semester 2/Learning Skills through Case Studies/Assignment 4/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Download weather data

In [ ]:
import urllib.request, json 
import datetime
import calendar

def downloadWeatherMonthly(key, in_year, in_month, in_day, end_year, end_month, end_day):

  year, month, day_1 = in_year, in_month, in_day

  while year < end_year or (year == end_year and month <= end_month):

    if year == end_year and month == end_month:
      day_2 = end_day
    else:
      day_2 = calendar.monthrange(year, month)[1]

    api_url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={key}&q=Leeds&format=json&date={year}-{str(month).zfill(2)}-{str(day_1).zfill(2)}&enddate={year}-{str(month).zfill(2)}-{str(day_2).zfill(2)}&tp=1"
    with urllib.request.urlopen(api_url) as url:
      data = json.loads(url.read().decode())
    with open(f'{root}Leeds_{year}-{str(month).zfill(2)}-{str(day_1).zfill(2)}_{year}-{str(month).zfill(2)}-{str(day_2).zfill(2)}.json', 'w') as f:
      json.dump(data, f)

    if month is 12:
      year += 1
      month = 1
    else:
      month += 1
    day_1 = 1



Set World Weather Online account and download data

In [ ]:
key = "###############################"

downloadWeatherMonthly(key, 2007, 7, 20, 2014, 1, 30)

Read CSV with cameras data

In [ ]:
from os import getcwd, listdir
from os.path import join, basename
from datetime import datetime
import glob
import pandas as pd

dateparse = lambda x: datetime.strptime(x, '%d/%m/%y')

df = pd.read_csv(f'{root}cameras_data.csv', sep=',', parse_dates=['Date'], date_parser=dateparse)
df = df.sort_values(by=['Date', 'Hour']).reset_index(drop=True)

print(df.head())
print(df.dtypes)

        Date   Hour LocationName  Count WeekDay  WeekNum Month  Year
0 2007-07-20  00:00     Briggate    251  Friday       29  July  2007
1 2007-07-20  01:00     Briggate    169  Friday       29  July  2007
2 2007-07-20  02:00     Briggate    173  Friday       29  July  2007
3 2007-07-20  03:00     Briggate     92  Friday       29  July  2007
4 2007-07-20  04:00     Briggate    114  Friday       29  July  2007
Date            datetime64[ns]
Hour                    object
LocationName            object
Count                    int64
WeekDay                 object
WeekNum                  int64
Month                   object
Year                     int64
dtype: object


Check data quality

In [ ]:
# Check if any empty value
print(f'Are there any empty value? {df.isnull().values.any()}')

# Create new df with summarised data
df_summary = pd.DataFrame(columns=['LocationName', 'First Date', 'Last Date', 'Missing days', 'Missing hours', 'Min', 'Mean', 'Med', 'Max'])
df_grouped = df.groupby(by=['LocationName'])

i = 0
for name, group in df_grouped:

    # Check number of days
    missing_days = 0
    df_grouped_day = group.groupby(by=['Date']).count()
    diff = group.Date.max() - group.Date.min()
    if diff.days != (df_grouped_day.shape[0] - 1):
        missing_days = diff.days
        print(f'Group "{name}" has missing days: \n   Expected days: {diff.days}\n   Actual days: {(df_grouped_day.shape[0] - 1)}')
    
    # Check if there are 24 records for each date available
    missing_hours = 0
    indexes = df_grouped_day.index[(df_grouped_day['Hour'] != 24)]
    if indexes.size > 0:
        missing_hours = indexes.size
        print(f'Group "{name}" has missing hours:')
        print(indexes.values)
    
    # Save summary
    df_summary.loc[i] = [name, group.Date.min(), group.Date.max(), missing_days, missing_hours, group.Count.min(), group.Count.mean(), group.Count.median(), group.Count.max()]
    i += 1

df_summary

Are there any empty value? False


,LocationName,First Date,Last Date,Missing days,Missing hours,Min,Mean,Med,Max
0,Albion Street North,2008-08-16,2014-01-30,0,0,0,837.627967,232.0,9317
1,Albion Street South,2008-08-16,2014-01-30,0,0,0,1207.419028,342.0,23268
2,Briggate,2007-07-20,2014-01-30,0,0,0,1269.573349,505.0,19820
3,Briggate at McDonalds\t,2008-06-27,2014-01-30,0,0,0,648.551553,301.0,5652
4,Commercial Street at Barratts,2009-07-17,2014-01-30,0,0,0,891.583158,259.0,10102
5,Commercial Street at Lush,2008-06-27,2014-01-30,0,0,0,1307.238095,238.0,19745
6,Dortmund Square\t,2008-08-27,2014-01-30,0,0,0,1009.154165,389.0,9007
7,Headrow,2008-08-27,2014-01-30,0,0,0,611.930850,275.0,5658


Create new excel with a different format for Tableau (only Date and Counts by Camera)

In [ ]:
import numpy as np

df_counts = df
df_counts['Date'] = df_counts.apply (lambda row: row['Date'].replace(hour=int(row['Hour'][:2])), axis=1)
df_counts.drop(['Hour', 'WeekDay', 'WeekNum', 'Month', 'Year'], axis=1, inplace=True)

# Create new df with summarised data
df_summary = pd.DataFrame(columns=['Date', 'Albion Street North', 'Albion Street South', 'Briggate', 'Briggate at McDonalds', 'Commercial Street at Barratts', 'Commercial Street at Lush', 'Dortmund Square', 'Headrow'])
df_grouped = df_counts.groupby(by=['Date'])
df_grouped.head()

def getCount(loc_name, df):
    index = df.index[(df['LocationName'] == loc_name)]
    if index.size > 0:
        return df.at[index[0], 'Count']
    else:
        return np.nan

i = 0
for name, group in df_grouped:
    df_summary.loc[i] = [name, getCount("Albion Street North", group), getCount("Albion Street South", group), getCount("Briggate", group), getCount("Briggate at McDonalds\t", group), 
                         getCount("Commercial Street at Barratts", group), getCount("Commercial Street at Lush", group), getCount("Dortmund Square\t", group), getCount("Headrow", group)]
    i += 1

print('Saving to basic excel...')
df_tosave = df_summary
df_tosave.to_excel(root + 'cameras_data_Tableau.xlsx', index=False)
print('Finished')

Saving to basic excel...
Finished


Aggregate weather data to cameras data

In [ ]:
import numpy as np
import json

# Add new columns
df["Temperature"] = np.nan
df["Wind"] = np.nan
df["Precipitation"] = np.nan
df["Humidity"] = np.nan
df["Cloud Cover"] = np.nan
# df.astype({"Temperature": int, "Wind": int, "Precipitation": float})

# List with all .json files
files = sorted(glob.glob(join(root + 'Weather Data/', "*.json")))

# Go through all weather files
for file in files:
    print(basename(file))
    json_file = open(file, encoding='iso8859')
    data = json.load(json_file)

    try:
        for data_date in data['data']['weather']:
            for data_hourly in data_date['hourly']:
                time = data_hourly['time'].zfill(4)
                time = time[:2] + ':' + time[2:]
                indexes = df.index[(df['Date'] == data_date['date']) & (df['Hour'] == time)]
                df.loc[indexes, ['Temperature', 'Wind', 'Precipitation', 'Humidity', 'Cloud Cover']] = [int(data_hourly['tempC']), int(data_hourly['windspeedKmph']), float(data_hourly['precipMM']), int(data_hourly['humidity']), int(data_hourly['cloudcover'])]
            # break

    except Exception as inst:
        print(inst)

    # break

Leeds_2007-07-20_2007-07-31.json
'weather'
Leeds_2007-08-01_2007-08-31.json
'weather'
Leeds_2007-09-01_2007-09-30.json
'weather'
Leeds_2007-10-01_2007-10-31.json
'weather'
Leeds_2007-11-01_2007-11-30.json
'weather'
Leeds_2007-12-01_2007-12-31.json
'weather'
Leeds_2008-01-01_2008-01-31.json
'weather'
Leeds_2008-02-01_2008-02-29.json
'weather'
Leeds_2008-03-01_2008-03-31.json
'weather'
Leeds_2008-04-01_2008-04-30.json
'weather'
Leeds_2008-05-01_2008-05-31.json
'weather'
Leeds_2008-06-01_2008-06-30.json
'weather'
Leeds_2008-07-01_2008-07-31.json
Leeds_2008-08-01_2008-08-31.json
Leeds_2008-09-01_2008-09-30.json
Leeds_2008-10-01_2008-10-31.json
Leeds_2008-11-01_2008-11-30.json
Leeds_2008-12-01_2008-12-31.json
Leeds_2009-01-01_2009-01-31.json
Leeds_2009-02-01_2009-02-28.json
Leeds_2009-03-01_2009-03-31.json
Leeds_2009-04-01_2009-04-30.json
Leeds_2009-05-01_2009-05-31.json
Leeds_2009-06-01_2009-06-30.json
Leeds_2009-07-01_2009-07-31.json
Leeds_2009-08-01_2009-08-31.json
Leeds_2009-09-01_2009-

In [ ]:
print('Saving to basic excel...')
df_tosave = df
df_tosave['Date'] = df_tosave.apply (lambda row: row['Date'].replace(hour=int(row['Hour'][:2])), axis=1)
# df_tosave.drop(['Hour'], axis=1, inplace=True)
df_tosave.to_excel(root + 'cameras_data_weather_FULL.xlsx', index=False)
print('Finished')

Saving to basic excel...
Finished


Change dates for fixtures

In [ ]:
def getHour(time):
    hour = time.hour
    if time.minute >= 30:
        hour += 1
    return hour

In [ ]:
print('Loading file')
df = pd.read_excel(f'{root}Fixtures.xlsx')
df_tosave = df
df_tosave['Date'] = df_tosave.apply (lambda row: row['Date'].replace(hour=getHour(row['Hour'])), axis=1)
df_tosave.drop(['Hour'], axis=1, inplace=True)
df_tosave.to_excel(root + 'Fixtures_Tableau.xlsx', index=False)
print('Finished')

Loading file
Finished


Create dummie variables

In [ ]:
df = pd.read_excel(f'{root}cameras_data_weather_FULL.xlsx')

cat_vars=['Hour', 'Month','WeekDay','LocationName']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var)
    data1=df.join(cat_list)
    df=data1

# cat_vars=['Hour', 'Month','WeekDay','LocationName']
# data_vars=df.columns.values.tolist()
# to_keep=[i for i in data_vars if i not in cat_vars]

# data_final=df[to_keep]
data_final=df
data_final.columns.values

array(['Date', 'Hour', 'LocationName', 'Count', 'WeekDay', 'WeekNum',
       'Month', 'Year', 'Temperature', 'Wind', 'Precipitation',
       'Humidity', 'Cloud Cover', 'Hour_00:00', 'Hour_01:00',
       'Hour_02:00', 'Hour_03:00', 'Hour_04:00', 'Hour_05:00',
       'Hour_06:00', 'Hour_07:00', 'Hour_08:00', 'Hour_09:00',
       'Hour_10:00', 'Hour_11:00', 'Hour_12:00', 'Hour_13:00',
       'Hour_14:00', 'Hour_15:00', 'Hour_16:00', 'Hour_17:00',
       'Hour_18:00', 'Hour_19:00', 'Hour_20:00', 'Hour_21:00',
       'Hour_22:00', 'Hour_23:00', 'Month_April', 'Month_August',
       'Month_December', 'Month_February', 'Month_January', 'Month_July',
       'Month_June', 'Month_March', 'Month_May', 'Month_November',
       'Month_October', 'Month_September', 'WeekDay_Friday',
       'WeekDay_Monday', 'WeekDay_Saturday', 'WeekDay_Sunday',
       'WeekDay_Thursday', 'WeekDay_Tuesday', 'WeekDay_Wednesday',
       'LocationName_Albion Street North',
       'LocationName_Albion Street South', 'Loca

In [ ]:
data_final.head()

,Date,Hour,LocationName,Count,WeekDay,WeekNum,Month,Year,Temperature,Wind,Precipitation,Humidity,Cloud Cover,Hour_00:00,Hour_01:00,Hour_02:00,Hour_03:00,Hour_04:00,Hour_05:00,Hour_06:00,Hour_07:00,Hour_08:00,Hour_09:00,Hour_10:00,Hour_11:00,Hour_12:00,Hour_13:00,Hour_14:00,Hour_15:00,Hour_16:00,Hour_17:00,Hour_18:00,Hour_19:00,Hour_20:00,Hour_21:00,Hour_22:00,Hour_23:00,Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,Month_May,Month_November,Month_October,Month_September,WeekDay_Friday,WeekDay_Monday,WeekDay_Saturday,WeekDay_Sunday,WeekDay_Thursday,WeekDay_Tuesday,WeekDay_Wednesday,LocationName_Albion Street North,LocationName_Albion Street South,LocationName_Briggate,LocationName_Briggate at McDonalds\t,LocationName_Commercial Street at Barratts,LocationName_Commercial Street at Lush,LocationName_Dortmund Square\t,LocationName_Headrow
0,2007-07-20 00:00:00,00:00,Briggate,251,Friday,29,July,2007,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,2007-07-20 01:00:00,01:00,Briggate,169,Friday,29,July,2007,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,2007-07-20 02:00:00,02:00,Briggate,173,Friday,29,July,2007,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,2007-07-20 03:00:00,03:00,Briggate,92,Friday,29,July,2007,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2007-07-20 04:00:00,04:00,Briggate,114,Friday,29,July,2007,NaN,NaN,NaN,NaN,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
print('Saving to basic excel...')
df_tosave = data_final
# df_tosave.drop(['Date'], axis=1, inplace=True)
df_tosave.to_excel(root + 'cameras_data_weather_model_FULL.xlsx', index=False)
print('Finished')

Saving to basic excel...
Finished


Check if there is any missing data

In [ ]:
df

,Date,Hour,LocationName,Count,WeekDay,WeekNum,Month,Year,Temperature,Wind,Precipitation
0,2007-07-20,00:00,Briggate,251,Friday,29,July,2007,NaN,NaN,NaN
1,2007-07-20,01:00,Briggate,169,Friday,29,July,2007,NaN,NaN,NaN
2,2007-07-20,02:00,Briggate,173,Friday,29,July,2007,NaN,NaN,NaN
3,2007-07-20,03:00,Briggate,92,Friday,29,July,2007,NaN,NaN,NaN
4,2007-07-20,04:00,Briggate,114,Friday,29,July,2007,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
386107,2014-01-30,23:00,Briggate,480,Thursday,5,January,2014,2.0,11.0,0.0
386108,2014-01-30,23:00,Commercial Street at Barratts,133,Thursday,5,January,2014,2.0,11.0,0.0
386109,2014-01-30,23:00,Commercial Street at Lush,128,Thursday,5,January,2014,2.0,11.0,0.0
386110,2014-01-30,23:00,Dortmund Square\t,139,Thursday,5,January,2014,2.0,11.0,0.0
